# Module 5 R Notebook

# Gene Alterations

## Important note - read first

In this course, you are not required to become proficient in writing R scripts for Big Biomedical Data Analytics. The assignments will be based on each module's template, and replacing some parts of them with new data and parameters so that you will get new results and interpret them. 

Understanding all the coding details in R will depend on your bakcground knowledge in programming concepts. 

Therefore if you do not understand all the coding details in R, try to run the code and understand mostly: 

1) what information do I have to provide to this script to make it work ?

2) how to execute this script (it will always be: run the cell containing the script Cell --> Run cells, or click on SHIFT + ENTER at the same time, or simply click on the right arrow icon) ?

3) what results do I get ?

4) how to interpret the results ?

Little by little, you will become familiar with R programming language and, example after example, you will understand the scripts and be ready for more advanced programming work. 

## Assignment goals

In this notebook, you are going to practice analyzing files containing copy number variations. Since for a known population, cBioPortal cna be used for that purpose, we are going to get reults at the patient level. 

Here we are continuing to work on a breast cancer dataset downloaded from Firehose (https://gdac.broadinstitute.org/). The dataset used is "BRCA.snp__genome_wide_snp_6__broad_mit_edu__Level_3__segmented_scna_minus_germline_cnv_hg19__seg.seg.txt". This is a complete dataset of copy number alterations from the whole geneome compared with a normal genome. 

This dataset has 6 columns:

         Sample                    Chromosome     Start       End        Num_Probes  Segment_Mean
1 TCGA-3C-AAAU-10A-01D-A41E-01          1        3218610      95674710      53225       0.0055
2 TCGA-3C-AAAU-10A-01D-A41E-01          1        95676511     95676518          2      -1.6636

'Sample' represents the patient ID (there are some normals too as we saw previously).
'Chromosome' represents the chromosome number.
'Start' represents the start position of the segmented window.
'End' represents the end position of the segmented window.
'Num_Probes' represents the number of probes in the segmented window.
'Segment_Mean' represents the mean copy numner estimate of this particular segment.

When 'Segment_Mean' is greater than 0, there is amplification, and when it is less than 0, there is deletion. However slight variations do not count much, as we saw. Often cut-off values of +0.2 and -0.2 are chosen to determine whether there is amplificaiton or deletion.

In this assignment, you will find whether a particular patient has copy number alterations, and how much.

So let us get started !

## Preparing the environment

Good news - we are not going to use any libraries for this assignment !

In [3]:
# no library to load



## Loading the data

We load the dataset downloaded from Firehose here. We check how many rows and columns and display some of the data to see what they look like. 

We are using the header row as variable names for the columns of our 'cnvlogs' dataframe, which is accomplished through 'header = T').

We see that we have 284,458 rows and 6 columns. 

Because we have only 6 columns, we can look at the first 6 rows using 'head'.

we notice right away that we have many rows for the same patient because each row corresponds to a particular probe, or location on the genome.

In [4]:
# cell #1
cnvLogs <- read.table("readonly/BRCA.snp__genome_wide_snp_6__broad_mit_edu__Level_3__segmented_scna_minus_germline_cnv_hg19__seg.seg.txt", 
            header = T, fill = T)
dim(cnvLogs)
# 284458 6
head(cnvlogs)
#             Sample Chromosome     Start       End Num_Probes Segment_Mean
#1 TCGA-3C-AAAU-10A-01D-A41E-01          1   3218610  95674710      53225       0.0055

[1] 284458      6

ERROR: Error in head(cnvlogs): object 'cnvlogs' not found


## Data understanding

To get a sense of whether our data overall is more amplified or deleted, we can run summary statistics.

We find that overall our data are more deleted since the mean is negative. However this should be used with caution because we may have a mix of normals and patients in this dataset.

In [5]:
# cell #2
summary(cnvLogs)
# mean = -0.1132 median = 0


                          Sample         Chromosome       Start          
 TCGA-BH-A0HQ-01A-11D-A036-01:  1678   Min.   : 1.0   Min.   :   284018  
 TCGA-A2-A04U-01A-11D-A111-01:  1602   1st Qu.: 5.0   1st Qu.: 17849617  
 TCGA-B6-A0X7-10A-01D-A10L-01:  1584   Median :10.0   Median : 50042220  
 TCGA-A8-A06R-01A-11D-A011-01:   968   Mean   :10.4   Mean   : 62737364  
 TCGA-A8-A06U-01A-11D-A011-01:   886   3rd Qu.:16.0   3rd Qu.: 95576350  
 TCGA-E9-A1NA-01A-11D-A141-01:   796   Max.   :23.0   Max.   :247812431  
 (Other)                     :276944                                     
      End              Num_Probes      Segment_Mean    
 Min.   :   305278   Min.   :     2   Min.   :-7.5883  
 1st Qu.: 39329031   1st Qu.:    42   1st Qu.:-0.3786  
 Median : 74057289   Median :  1018   Median : 0.0010  
 Mean   : 85074001   Mean   : 12064   Mean   :-0.1132  
 3rd Qu.:123232246   3rd Qu.: 13112   3rd Qu.: 0.2368  
 Max.   :247813706   Max.   :131334   Max.   : 5.7732  
                

## Copy number alterations of a single patient

Now for a particular patient, say "TCGA-A8-A06R-01A-11D-A011-01", we are going to:

1) find how many potential copy number variations he/she has (just counting how many rows he/she has in this dataset).

2) find how many amplifications ('Segment_Mean' > 0 ) he/she has.

3) find how many deletions ('Segment_Mean' > 0 ) he/she has.

4) find the average 'Segment_Mean' he/she has to get a global picture. 

So first, we count how many rows he/she has in this dataset. this can be accomplished using a 'subset' function in R to select all the rows for this patient, then we count how mnay we have using 'nrow'. 

We find 968.


In [6]:
# cell #3
nrow(subset(cnvLogs, Sample == "TCGA-3C-AAAU-01A-11D-A41E-01"))

[1] 313

Next, we calculate how many of these rows have 'Segment_Mean' > 0.

We find 585.

In [7]:
# cell #4
nrow(subset(cnvLogs, Sample == "TCGA-3C-AAAU-01A-11D-A41E-01" & Segment_Mean > 0))

[1] 207

Similarly, we calculate how many of these rows have 'Segment_Mean' < 0.

We find 383.

In [9]:
# cell #5
nrow(subset(cnvLogs, Sample == "TCGA-3C-AAAU-01A-11D-A41E-01" & Segment_Mean < 0))

[1] 106

Finally, we calculate the mean of 'Segment_Mean' for this patient. 

We find a positive value of 0.2, which is above the amplification threshold.

In [10]:
# cell #6
mean(subset(cnvLogs, Sample == "TCGA-3C-AAAU-01A-11D-A41E-01")[["Segment_Mean"]])

[1] 0.3589546

We can say that this patient has 968 copy number alterations, with 585 amplifications and 383 deletions. Overall the patient has more amplifications than deletions, which is confirmed by his overall score of 0.2. 

It would be interesting to calculate globally the same figures for all normals and all tumor patients.